# Train and deploy on Google Cloud Platform (GCP)

This notebook introduces you to using Kubeflow Fairing to train and deploy a model to Kubeflow on Google Kubernetes Engine (GKE), and Google Cloud ML Engine. This notebook demonstrate how to:
 
* Train an XGBoost model in a local notebook,
* Use Kubeflow Fairing to train an XGBoost model remotely on Kubeflow,
* Use Kubeflow Fairing to train an XGBoost model remotely on Cloud ML Engine,
* Use Kubeflow Fairing to deploy a trained model to Kubeflow, and
* Call the deployed endpoint for predictions.

To learn more about how to run this notebook locally, see the guide to [training and deploying on GCP from a local notebook][gcp-local-notebook].

[gcp-local-notebook]: https://kubeflow.org/docs/fairing/gcp-local-notebook/

In [7]:
import json
import requests
import numpy as np
import os
from pathlib import Path
import pprint
import importlib
import logging
import sys

logging.basicConfig(format='%(message)s')
logging.getLogger().setLevel(logging.INFO)

fairing_code = os.path.join(Path.home(), "git_kubeflow-fairing")

if os.path.exists(fairing_code):    
    logging.info("Adding %s to path", fairing_code)
    sys.path = [fairing_code] + sys.path

import fairing
from fairing import cloud

Adding /home/jovyan/git_kubeflow-fairing to path


In [8]:
# Setting up google container repositories (GCR) for storing output containers
# You can use any docker container registry istead of GCR
GCP_PROJECT = cloud.gcp.guess_project_name()
DOCKER_REGISTRY = 'gcr.io/{}/fairing-job'.format(GCP_PROJECT)
BASE_IMAGE = "gcr.io/code-search-demo/mlapp/base:v20190410-ea12733-dirty-23b869"

logging.info("PROJECT=%s", GCP_PROJECT)
logging.info("DOCKER_REGISTRY=%s", DOCKER_REGISTRY)
logging.info("BASE_IMAGE=%s", BASE_IMAGE)

PROJECT=code-search-demo
DOCKER_REGISTRY=gcr.io/code-search-demo/fairing-job
BASE_IMAGE=gcr.io/code-search-demo/mlapp/base:v20190410-ea12733-dirty-23b869


## Deploy on Kubeflow

Import the `fairing` library and configure the GCP environment that your training or prediction job will run in.

In [9]:
import deploy_with_fairing
importlib.reload(deploy_with_fairing)
_, _, deployer = deploy_with_fairing.deploy(DOCKER_REGISTRY, BASE_IMAGE)

Using context dir /tmp/tmpru3rn1fh
Using preprocessor: <fairing.preprocessors.base.BasePreProcessor object at 0x7f6154f5ef28>
Using builder: <fairing.builders.append.append.AppendBuilder object at 0x7f6126d40320>
Building image...
Creating docker context: /tmp/fairing.context.tar.gz
Adding files to context: {'mlapp.py', 'LabelPrediction.py', 'sql_models.py', 'utils.py', 'app.py', 'deploy_with_fairing.py'}
Context: /tmp/fairing.context.tar.gz, Adding /home/jovyan/git_kubeflow-fairing/fairing/__init__.py at /app/fairing/__init__.py
Context: /tmp/fairing.context.tar.gz, Adding /home/jovyan/git_kubeflow-fairing/fairing/runtime_config.py at /app/fairing/runtime_config.py
Context: /tmp/fairing.context.tar.gz, Adding mlapp.py at /app/mlapp.py
Context: /tmp/fairing.context.tar.gz, Adding LabelPrediction.py at /app/LabelPrediction.py
Context: /tmp/fairing.context.tar.gz, Adding sql_models.py at /app/sql_models.py
Context: /tmp/fairing.context.tar.gz, Adding utils.py at /app/utils.py
Context: /t

## Call the prediction endpoint

Create a test dataset, then call the endpoint on Kubeflow for predictions.

In [10]:
def predict(url, data, feature_names=None):
        pdata={
            "data": {
                "names":feature_names,
                "tensor": {
                    "shape": np.asarray(data.shape).tolist(),
                    "values": data.flatten().tolist(),
                },
            }
        }
        serialized_data = json.dumps(pdata)
        r = requests.post(url, data={'json':serialized_data})
        return r

In [13]:
# Note: first prediction is slow because we are loading the model.
url = "http://{0}.{1}.svc.cluster.local:5000/predict".format(
    deployer.service.metadata.name, deployer.service.metadata.namespace)
    
body = "The cluster app is completely broken. Please fix it immediately."
title = "The cluster app is not working; please fix"  
r = predict(url, np.array([title, body]))
pprint.pprint(r.content)


(b'{"data":{"names":["t:0","t:1","t:2"],"tensor":{"shape":[1,3],"values":[0.948'
 b'5112428665161,0.04742114990949631,0.004067644942551851]}},"meta":{}}\n')


In [ ]:
# Cleanup
!kubectl delete service -l app=mlapp
!kubectl delete deploy -l app=mlapp